# LFortran: Interactive Fortran Compiler

This is a Demo notebook that was [originally used](https://nbviewer.jupyter.org/gist/certik/f1d28a486510810d824869ab0c491b1c) with the Python prototype of LFortran. The new C++ production implementation can now execute all the cells in the original notebook (and more).


## Calculator

In [18]:
1+3

4

In [19]:
(2+3)*3

15

## Global Variables

In [20]:
integer :: i

LFortran Exception: addModule() returned an error: Duplicate definition of symbol 'i'

In [21]:
integer :: j

LFortran Exception: addModule() returned an error: Duplicate definition of symbol 'j'

In [22]:
i = 5
j = 6

In [23]:
i

5

In [24]:
(i+j)*3

33

## Functions

You can declare a function and use it in expressions:

In [25]:
integer function fn(a, b)
integer, intent(in) :: a, b
fn = a + b
end function

LFortran Exception: addModule() returned an error: Duplicate definition of symbol 'fn'

In [26]:
fn(2, 3)

5

In [27]:
fn(2, 3)*3

15

You can redeclare a function and it will shadow the old `fn` function. Now `fn(2, 3)` returns a different value:

In [28]:
integer function fn2(a, b)
integer, intent(in) :: a, b
fn2 = a - b
end function

fn2(2, 3)

LFortran Exception: addModule() returned an error: Duplicate definition of symbol 'fn2'

## Control Flow, Printing

You can use loops, if statements or the `print` function.

In [29]:
integer :: k
do k = 1, 4
    if (k == 3) cycle
    print *, "variable i =", k
end do

LFortran Exception: addModule() returned an error: Duplicate definition of symbol 'k'

# Plotting

We are designing how plotting should work in [#209](https://gitlab.com/lfortran/lfortran/-/issues/209). So this prototype code is now commented out:

In [30]:
!integer :: i, tmp
!do i = 1, 4
!    tmp = plot(1, i+1, 1)
!end do
!show()

# Magic Commands

### Compiler

In [31]:
integer :: n
n = 5
j = 0

LFortran Exception: addModule() returned an error: Duplicate definition of symbol 'n'

Show Abstract Syntax Tree (AST) after parsing (based on syntax only, no semantics):

In [32]:
%%showast
do i = 1, n
    j = j + i
end do

(TranslationUnit [(DoLoop 0 () 0 i 1 n () [(= 0 j (+ j i) ())] () ())])

Show LLVM code:

In [33]:
%%showllvm
do i = 1, n
    j = j + i
end do

; ModuleID = 'LFortran'
source_filename = "LFortran"

@i = external global i32
@j = external global i32
@k = external global i32
@n = external global i32

define void @__lfortran_evaluate_31() {
.entry:
  store i32 0, i32* @i, align 4
  br label %loop.head

loop.head:                                        ; preds = %loop.body, %.entry
  %0 = load i32, i32* @i, align 4
  %1 = add i32 %0, 1
  %2 = load i32, i32* @n, align 4
  %3 = icmp sle i32 %1, %2
  br i1 %3, label %loop.body, label %loop.end

loop.body:                                        ; preds = %loop.head
  %4 = load i32, i32* @i, align 4
  %5 = add i32 %4, 1
  store i32 %5, i32* @i, align 4
  %6 = load i32, i32* @j, align 4
  %7 = load i32, i32* @i, align 4
  %8 = add i32 %6, %7
  store i32 %8, i32* @j, align 4
  br label %loop.head

loop.end:                                         ; preds = %loop.head
  br label %return

return:                                           ; preds = %loop.end
  ret void
}

declare i32 @fn(i32

Show assembly code:

In [34]:
%%showasm
do i = 1, n
    j = j + i
end do

	.text
	.file	"LFortran"
	.globl	__lfortran_evaluate_32
	.p2align	4, 0x90
	.type	__lfortran_evaluate_32,@function
__lfortran_evaluate_32:
	.cfi_startproc
	movl	$0, i(%rip)
	.p2align	4, 0x90
.LBB0_1:
	movl	i(%rip), %eax
	incl	%eax
	cmpl	n(%rip), %eax
	jg	.LBB0_3
	movl	i(%rip), %eax
	incl	%eax
	movl	%eax, i(%rip)
	addl	%eax, j(%rip)
	jmp	.LBB0_1
.LBB0_3:
	retq
.Lfunc_end0:
	.size	__lfortran_evaluate_32, .Lfunc_end0-__lfortran_evaluate_32
	.cfi_endproc

	.section	".note.GNU-stack","",@progbits
